In [3]:
import os

In [ ]:
#################################
#################################
################################

In [15]:
# Folder holding the BraTS samples

# ONE BELOW IS THE MAIN ONE USED
# source_dirpath = '/raid/datasets/BraTS22/BraTS2022_Training/RSNA_ASNR_MICCAI_BraTS2021_TrainingData_16July2021'
source_dirpath = '/raid/edwardsb/projects/RANO/FAKE_withlabel3_ExampleBraTS22_Labels0-4'

# destination folder to hold the medperf postopp style data format
# (should be cleared of all items before re-running if already have run once)


# ONE BELOW IS MAIN ONE USED
# postopp_pardir = '/raid/edwardsb/projects/RANO/test_data_links_micahtest'
postopp_pardir = '/raid/edwardsb/projects/RANO/test_data_links_testforhasan'

# total number of samples to use
total_samples = 10

In [16]:


"""
Creating some data links that mimic the folder structure expected after data prep

"""


src_mods = ['t1.nii.gz', 't2.nii.gz', 't1ce.nii.gz', 'flair.nii.gz', 'seg.nii.gz']
dst_mods = ['t1n.nii.gz', 't2w.nii.gz', 't1c.nii.gz', 't2f.nii.gz', 'final_seg.nii.gz']

patients = list(os.listdir(source_dirpath))[:total_samples]
for src_mod, dst_mod in zip(src_mods, dst_mods):
    if src_mod == 'seg.nii.gz':
        subdir = 'labels'
    else:
        subdir = 'data'
    
    for patient in patients:
        dst_pat_dir = os.path.join(postopp_pardir, subdir, patient)
        if not os.path.exists(dst_pat_dir):
            os.makedirs(dst_pat_dir)
        src_pat_dir = os.path.join(source_dirpath, patient)
        for timestamp in ['2008.03.88', '2008.12.99']:
            dst_pattim_dir = os.path.join(dst_pat_dir, timestamp)
            if not os.path.exists(dst_pattim_dir):
                os.makedirs(dst_pattim_dir)
       
            


            src_path = os.path.join(src_pat_dir, patient + '_' + src_mod)
            if dst_mod == 'final_seg.nii.gz':
                dst_path = os.path.join(dst_pattim_dir, patient + '_' + timestamp + '_' + dst_mod)
            else:
                dst_path = os.path.join(dst_pattim_dir, patient + '_' + timestamp + '_brain_' + dst_mod)
            os.symlink(src=src_path, dst=dst_path)
        

